# Using PyTorch

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import sklearn.metrics as metrics

In [2]:
BATCH_SIZE = 32

transform = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root="./data", train = True, download = True, transform = transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 2)

testset = torchvision.datasets.MNIST(root = "./data", train = False, download = True, transform = transform)

testloader = torch.utils.data.DataLoader(testset, batch_size = BATCH_SIZE, shuffle = False, num_workers = 2)

In [3]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3)
        self.d1 = nn.Linear(26 * 26 * 32, 128)
        self.d2 = nn.Linear(128, 10)

    def forward(self, x):
        # 32x1x28x28 -> 32x32x26x26
        x = self.conv1(x)
        x = F.relu(x)
        
        #Flatten -> 32 x (32x26x26)
        x = x.flatten(start_dim = 1)

        #32 x (32x26x26) -> 32*128
        x = self.d1(x)
        x = F.relu(x)

        #logits -> 32, 10
        logits = self.d2(x)
        out = F.softmax(logits, dim = 1)
        return out

In [10]:
learning_rate = 0.001
num_epochs = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = MyModel()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
for epoch in range(num_epochs):
    train_running_loss = 0.0
    train_acc = 0.0

    for i, (images, labels) in enumerate(trainloader):

        images = images.to(device)
        labels = labels.to(device)

        logits = model(images)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        train_running_loss += loss.detach().item()
        train_acc += (torch.argmax(logits, 1).flatten() == labels).type(torch.float).mean().item()

    print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
          %(epoch, train_running_loss / i, train_acc/i))

Epoch: 0 | Loss: 1.6153 | Train Accuracy: 0.85
Epoch: 1 | Loss: 1.4955 | Train Accuracy: 0.97
Epoch: 2 | Loss: 1.4840 | Train Accuracy: 0.98
Epoch: 3 | Loss: 1.4784 | Train Accuracy: 0.98
Epoch: 4 | Loss: 1.4751 | Train Accuracy: 0.99


In [12]:
test_acc = 0.0

for i, (images, labels) in enumerate(testloader, 0):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    test_acc += (torch.argmax(outputs, 1).flatten() == labels).type(torch.float).mean().item()

print("Test Accuracy: %.2f"%(test_acc/i))

Test Accuracy: 0.98


# Using PyTorch Geometric

In [14]:
!pip install  torch-scatter
!pip install  torch-sparse
!pip install  torch-cluster
!pip install pytorch-geometric
!pip install tensorboardX

  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp312-cp312-macosx_14_0_arm64.whl size=299902 sha256=4d48613f8fef405051e4bc51d4150581a4f77d597e9662c44fdc2454bc054f08
  Stored in directory: /Users/ishaanpaul/Library/Caches/pip/wheels/84/20/50/44800723f57cd798630e77b3ec83bc80bd26a1e3dc3a672ef5
Successfully built torch-scatter
  Preparing metadata (setup.py) ... done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp312-cp312-macosx_14_0_arm64.whl size=472208 sha256=7db4ed18ee4fc02481d1f77e20e1e5310b4996e644ff869351ce3ac938c6e209
  Stored in directory: /Users/ishaanpaul/Library/Caches/pip/wheels/71/fa/21/bd1d78ce1629aec4ecc924a63b82f6949dda484b6321eac6f2
Successfully built torch-sparse
  Preparing metadata (setup.py) ... done
  Created wheel for torch-cluster: filename=torch_cluster-1.6.3-cp312-cp312-macosx_14_0_arm64.whl size=288605 sha256=401e2f04faa9eec8e37358c88957e619cd258d4025bc7baddc3bcae209d7fb01
  Stored in d

In [16]:
import torch
import torch.nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

import time
from datetime import datetime

import networkx as nx
import numpy as np
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.transforms as T

from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [ ]:
class GNNStack(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, task='node'):
        super(GNNStack, self).__init__()
        self.task = task
        self.convs = nn.ModuleList()
        self.convs.append(self.build_conv_model(input_dim, hidden_dim))
        for l in range(2):
            self.convs.append(self.build_conv_model(hidden_dim, hidden_dim))
        
        #Post-message-passing
        self.post_mp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim), nn.Dropout(0.25),
            nn.Linear(hidden_dim, output_dim))
        if not(self.task == 'node' or self.task == 'graph'): 
            raise RuntimeError('Unknown task.')
        
        self.dropout = 0.25
        self.num_layers = 3

    def build_conv_model(self, input_dim, hidden_dim):
        if self.task == 'node':
            return pyg_nn.GCNConv(input_dim, hidden_dim)
        else:
            return pyg_nn.GINConv(nn.Sequential(nn.Linear(input_dim, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, hidden_dim)))
        
    def 
        